In [ ]:
# Complete tinyGPT
# Then build and train gpt2
# Papers - Attention is all you need
# GPT 2/3 paper

# Llamac/llama2.c
# Cuda 

#### commands
""""
To see the gpu stats
nvidia-smi 
# To see al the cpu stats


"""

In [16]:
# how to?
# Model arcthitecture -> 
# Data(find and filter) -> 
# train -> 
# testing/benchmarks -> 
# Sample
# Some other evaluations and researches like scaling laws, transformer sizing
# Configurator
# Play with some opensource model also
# pretraining | post training

In [17]:
from transformers import GPT2LMHeadModel

In [18]:
# model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
# sd_hf = model_hf.state_dict()

# for k, v in sd_hf.items():
#     print(k, v.shape)


In [19]:
# we train model from as small as one to context length
# so model will see all kinds of context length and from 1 to length of context(block size)
# when predicting the character
# batch - for parallel processing
# In multidimensional matrix
# Each dimension is axis and each axis has index
#dim=0 → collapsing rows → gives a summary per column
# dim=1 → collapsing columns → gives a summary per row
# [a, b, c] = [a][b][c] : same in slicing

In [ ]:
# B - Batch - parallel input sequence
# T - Time(vocab size, sequence length, memory used in prediction, Context size) - position of tokens
# c - channel(embedding dimensions, d_model) - Contains feature vector for each token
# torch.tril(torch.ones(T, T))
# embedding table contains feature vector for each token
# each row corresponds to a token and columns feature vector
# embedding table dimensions - (vocab_size, Channel)
# Each token becomes three vectors: 
# Query: what this token is looking for?
# Key: what this token has to offer
# Value: what the information this token contains for sharing to next token i thinks
# Encoder model: Takes all the tokens in consideration
# Decoder model: Takes only current and past tokens in consideration
# Query @ Key = Scores
# Scale - Stabilize scores
# Softmax(scores) = weights 
# Weighted average of values - New token representations
# Scores: strong positive value - strong match high attention
# Near zero: No match - Low attention
# Negative number: Disimilar - very low ignored after softmax
# Linear Layer is simple matrix multiplication- input @ weight + bias = w @ x + b
# Training decides that what query, key and value will contain
# Mainly backpropagation and gradient descent this is amazing
# embedding dimensions or channel dimensions or feature vector of a token
# is broken down in to key, query and vector and we train these three things
# and after training these things contains meaning representation of embedding dimensions
# query of current token, and key and values of all the past token
# Keep the dimensions of k, q, v so dot products should be easy
# Mostly same or half
# in multi head attention keep the dimension as embd_d // head_size
#  The dot product
# the dot prodcut in between |A||B|cos(theta)
# In Other words: Projection of a on b
# IOW: how much A aligns with the B
# iow: how much a is in the direction of B
# iow: similarity between two vectors
# Normalization means: mean 0 and Standard deviation 1
# Be award of exploding and vanishing gradient

import torch
B, T, C = 4, 8, 32 # consider this as stacked sheet of paper
x = torch.randn(B, T, C)
key = nn.Linear(32, 16)
print(key.weight.shape)
query = nn.Linear(32, 16)
k = key(x) # 4, 8, 16
q = query(x) # 4, 8, 16
out = (k @ q.transpose(-2, -1)) / k.shape[-1] ** 0.5 # (4, 8, 16) @ (4, 16, 8) -> 4, 8, 8
print(out.shape)
ones = torch.tril(torch.ones(T, T))
out = out.masked_fill(ones == 0, float("-inf"))
out
# print(key(x).shape) # 4, 8, 32 @ 4, 32, 16 ---> 4, 8, 16 
# ex = nn.Linear(3, 5)
# print(x2.weight)
# x11 = x1(x)
# y11 = x2(x)
# y11.shape
# x11 @ y11

torch.Size([16, 32])
torch.Size([4, 8, 8])


tensor([[[ 0.6065,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
             -inf],
         [ 0.6065,  0.9519,    -inf,    -inf,    -inf,    -inf,    -inf,
             -inf],
         [-0.0893, -0.6017, -0.2983,    -inf,    -inf,    -inf,    -inf,
             -inf],
         [ 0.1335,  0.0954,  0.2236,  0.0426,    -inf,    -inf,    -inf,
             -inf],
         [ 0.1996,  0.1531,  0.4165, -0.1471, -0.0293,    -inf,    -inf,
             -inf],
         [ 0.0801,  0.0950,  0.0096,  0.1158, -0.2239, -0.2265,    -inf,
             -inf],
         [-0.3166,  0.1213,  0.2842, -0.2970, -0.3716, -0.5273,  0.0706,
             -inf],
         [-0.0362, -0.4743, -0.4011,  0.1258,  0.0207,  0.0619, -0.1340,
          -0.0080]],

        [[ 0.6808,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
             -inf],
         [ 0.7927,  0.1183,    -inf,    -inf,    -inf,    -inf,    -inf,
             -inf],
         [-0.6759,  0.0339, -0.1183,    -inf,    -inf,    -inf,    -

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
B, T, C = 4, 8, 32
vocab_size = 64 # How many tokens?
batch_size = 4 # input sequence we will processed in parallel
block_size = 8 # Context length, Time dimension
embd_d = 32 # Feature vector to represent a token
head_size = 16 # size of head
dropout = 0.2
x = torch.randn(B, T, C) # Input data

# One Head Attention Block
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embd_d, head_size, bias=False) # y = x @ wT + b
        self.query = nn.Linear(embd_d, head_size, bias=False)
        self.value = nn.Linear(embd_d, head_size, bias=False)
        # to make lower triangle that will help in weighted average calculation
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size))) 
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        q = self.query(x) # (B, T, head_size)
        k = self.key(x) # (B, T, head_size)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** 0.5 # (B, T, head_size) @ (B, head_size) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf")) # (B, T, T), deactivating next tokens
        wei = F.softmax(wei, dim= -1) # (B, T, T)
        # helps NN to choose differnet set of Node in different times
        # helps to regularize the model so it doesn't overfit
        wei = self.dropout(wei) # this is form of regularization
        # Weighted aggregation of the values
        v = self.value(x) # (B, T, head_size)
        out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out
attention = Head(head_size)
y = attention(x)
print(y.shape)
print(y[0])

torch.Size([4, 8, 16])
tensor([[ 0.6919,  0.2581, -0.1701, -0.3947,  0.0910, -0.0118,  0.2231, -1.2307,
         -0.0039, -0.6741,  0.1459, -1.2867, -0.3197,  0.4475,  0.4934, -0.8440],
        [ 0.6919,  0.2581, -0.1701, -0.3947,  0.0910, -0.0118,  0.2231, -1.2307,
         -0.0039, -0.6741,  0.1459, -1.2867, -0.3197,  0.4475,  0.4934, -0.8440],
        [-0.0819, -0.3501, -0.2412,  0.0528,  0.2045,  0.7295, -0.0119, -0.0118,
         -0.4419,  0.2094,  1.0352, -0.0915,  0.4527, -0.6376, -0.2764,  0.0615],
        [-0.0855, -0.3502, -0.2420,  0.0576,  0.2039,  0.7239, -0.0121, -0.0083,
         -0.4364,  0.2113,  1.0321, -0.0880,  0.4440, -0.6383, -0.2797,  0.0601],
        [-0.3931, -0.5869, -0.2843,  0.1258,  0.2984,  0.1196,  0.4607,  0.4319,
          0.7149, -0.3157, -1.0503,  0.7572,  0.0563, -0.2047, -0.3196, -0.5844],
        [ 0.2483,  0.1578, -0.3307, -0.3907,  0.1050, -0.1453,  0.1301, -0.4667,
          0.2799, -0.6968, -0.1358, -0.1893, -0.0469,  0.3229,  0.0722, -0.5208],

In [ ]:
# Multi Head attention block
# 
embd_d = 32
num_heads = 4
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embd_d)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # one by one call each head, pass the data and then call the forward method
        # then concat the result
        out = torch.cat([h(x) for h in self.heads])
        # converth output to normal dimensions and apply dropout
        out = self.dropout(self.proj(out)) 
        return out


class FeedForward(nn.Module):
    def __init__(self, embd_d):
        super().__init__()
        self.net = nn.Sequential(
        nn.Linear(embd_d, 4 * embd_d),
        nn.ReLU(),
        nn.Linear(4 * embd_d, embd_d),
        nn.dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# whole decoder architecture
# 

class Block(nn.Module):
    def __init__(self):
        super().__init__(embd_d, num_heads)
        head_size = embd_d // num_heads
        self.s_attention = MultiHeadAttention(num_heads, head_size)
        self.feed_forward = FeedForward(embd_d)
        # layer norm normalizes the data with 0 mean and 1 standard deviation
        self.ln1 = nn.LayerNorm(embd_d)
        self.ln2 = nn.LayerNorm(embd_d)
    
    def forward(self, x):
       # Paper section 3.1
       # Adding result back to x is called residual connection or skip connection
       # Just add back don't replace it
       # Utilize the self attention and add back the result
       # it get called the residual connection because it's one part skips the transformation layer
       # and gets added in the output
       x = x + self.s_attention(self.ln1(x)) # pre norm for better result
       # then think for a sec feed forward and back the result
       x = x + self.feed_forward(self.ln2(x))
       return x


In [ ]:
# Final class GPT class
n_layer = 6
device = "cuda"
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Contains the meaning of the token
        self.token_embedding_table = nn.Embedding(vocab_size, embd_d)
        # Contains the position vector means this token is at this position and gives the position vector
        # helps model understand the position impact of the token
        # because it changes the meaning of token based on where they appear
        self.position_embedding_table = nn.Embedding(block_size, embd_d)
        self.blocks = nn.Sequential(*[Block(embd_d, num_heads) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embd_d)
        self.lm_head = nn.Linear(embd_d, vocab_size)
        self.apply(self._init_weights)

        # implement _init_weights

        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

        def forward(self, idx, target=None):
            B, T = idx.shape
            tok_emb = self.token_embedding_table(idx) # (B, T, C)
            pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
            x = tok_emb + pos_emb # (B, T, C)
            x = self.blocks(x) # (B, T, C)
            x = self.ln_f(x) # (B, T, C)
            logits = self.lm_head(x) # (B, T, vocab_size)
            if target is None:
                loss = None
            else:
                B, T, C = logits.shape
                logits = logits.view(B*T, C)
                targets = targets.view(B*T)
                loss= F.cross_entropy(logits, target)
            return logits, loss
        
        def generate(self, idx, max_new_tokens):

            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, loss = self(idx)
                # Get the last time step
                logits = [:, -1, :] # (B, C)
                # calculate the probablity using softmax
                prob = F.softmax(logits, dim=-1) # probality using channel dimensions
                # because that dimension is the feature vector

                idx_next = torch.multinomial(prob, num_samples=1) # (B, 1)

                idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)

            return idx